In [ ]:
import warnings;
warnings.filterwarnings('ignore');

In [ ]:
# 
import matplotlib.pyplot as plt
import pandas as pd
meDat = pd.read_csv('/home/or/kpe_task_analysis/task_based_analysis/kpe_sub_condition.csv')


In [ ]:
meDat

In [ ]:

allDat = pd.read_csv('/home/or/Documents/kpe_analyses/KPEIHR0009_DATA_2020-08-31_1301.csv')
pclDat = allDat.filter(like='pcl5')
# remove two irrelevant vars
del pclDat['pcl5_complete']
del pclDat['pcl5_total']
# sum all pcl scores
a = pclDat.sum(axis = 1, skipna = True) 
# add a to df
pclDat.insert(20, "pcl_total", a)
pclDat.insert(0, "scr_id", allDat['scr_id'])
pclDat.insert(1, "redcap_event_name", allDat["redcap_event_name"])
newPclDat = pclDat[((pclDat['redcap_event_name'] == 'screening_selfrepo_arm_1') | 
                    (pclDat['redcap_event_name'] == 'visit_1_arm_1') | 
                    #(pclDat['redcap_event_name'] == 'visit_2_infusion_s_arm_1') | 
                    (pclDat['redcap_event_name'] == 'visit_7_week_follo_arm_1') | 
                    (pclDat['redcap_event_name'] == '30_day_follow_up_s_arm_1') | 
                    (pclDat['redcap_event_name'] == '90_day_follow_up_s_arm_1'))]


In [ ]:
newPclDat.head()

In [ ]:
# remove space from scr_id
newPclDat['scr_id'] = newPclDat['scr_id'].str.replace(" ","")


In [ ]:
widePclDat = newPclDat.pivot(index='scr_id', columns='redcap_event_name', values='pcl_total')
widePclDat

In [ ]:
# remove all NAs in visit7
PclDat_nonNa = widePclDat.dropna(subset=['visit_7_week_follo_arm_1'])
# check if there is a 0 and change to NaN
#
PclDat_nonNa

In [ ]:
# rename columns
# merging datasets to have medical conditions also
fullPCL = PclDat_nonNa.merge(meDat, left_on='scr_id', right_on='scr_id', how='outer')

# rename
fullPCL = fullPCL.rename(columns={"30_day_follow_up_s_arm_1": "30_days", "90_day_follow_up_s_arm_1": "90_days",
                   "screening_selfrepo_arm_1":"screening", 
                   "visit_1_arm_1" : "visit_1",
                   "visit_7_week_follo_arm_1" : "visit_7"
                  })

# reorder columns
colOrder = ['scr_id','med_cond','screening','visit_1','visit_7','30_days', '90_days']
fullPCL = fullPCL[colOrder]
fullPCL

In [ ]:
## replace visit 1 missing values or zeroes with screening scores
#df.loc[df.ID == 103, 'FirstName'] = "Matt"
fullPCL.loc[fullPCL.scr_id=='KPE1339','visit_1'] = fullPCL.loc[fullPCL.scr_id=='KPE1339'].screening
fullPCL.loc[fullPCL.scr_id=='KPE1390','visit_1'] = fullPCL.loc[fullPCL.scr_id=='KPE1390'].screening
fullPCL.loc[fullPCL.scr_id=='KPE1464','visit_1'] = fullPCL.loc[fullPCL.scr_id=='KPE1464'].screening

In [ ]:
fullPCL = fullPCL[['scr_id','med_cond','visit_1','visit_7','30_days', '90_days']]

In [ ]:
fullPCL_melt = pd.melt(fullPCL, id_vars=['scr_id','med_cond'], var_name = 'time', value_name='pcl')
fullPCL_melt["time"] = fullPCL_melt.time.astype('category')
# save as csv
fullPCL_melt.to_csv('fullPCL_melt.csv', index = False)
fullPCL.to_csv('fullPCL.csv', index = False)
fullPCL_melt

In [ ]:
fullPCL = fullPCL.rename(columns={"30_days":"FollowUp_30", "90_days": "FollowUp90",
                   "visit_1": "Before_Treatmeant",
                   "visit_7": "After_Treatment"
                  })
fullPCL_melt = pd.melt(fullPCL, id_vars=['scr_id','med_cond'], var_name = 'time', value_name='pcl')
import seaborn as sns
from matplotlib.pyplot import figure
plt.figure(figsize=(10,8))
plt.title("Changes in PCL scores")
g1 = sns.boxplot(x='time',y='pcl', data=fullPCL_melt, showfliers = False)
sns.stripplot(x='time',y='pcl',hue='med_cond', data=fullPCL_melt, alpha=0.8, size=7)
plt.hlines(y = 33, xmin = -0.5, xmax=3.5, color="blue", linestyles="dashed")
g1.set_xlabel("")
g1.set_xticklabels(["Before Treatment", 'After Treatment', '30 Days Follow-Up', '90 Days Follow-Up'],
                  fontsize = 14)
g1.set_ylabel("Symptoms (PCL-5 Score)", fontsize = 14)
plt.show()



In [ ]:
fullPCL[['med_cond', 'visit_7',  '90_days']].groupby('med_cond').describe()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()
# px.strip(x='time', y='pcl', 
       #    data_frame=fullPCL_melt, color='med_cond')
px.box(x='time', y='pcl',
           data_frame=fullPCL_melt, color='med_cond')

In [ ]:
px.line(x='time', y='pcl', facet_col= 'med_cond', 
           data_frame=fullPCL_melt, color='scr_id')

In [ ]:
# create slope for each subject
changeDat = fullPCL[['scr_id','med_cond','visit_1','visit_7','30_days','90_days']]
changeDat['beforeTrt'] = changeDat.visit_1- changeDat.visit_1
changeDat['afterTrt'] = changeDat.visit_7- changeDat.visit_1
changeDat['monthFU'] = changeDat['30_days']- changeDat.visit_1
changeDat['ThreeMonths'] = changeDat['90_days'] - changeDat.visit_1

changeDat

In [ ]:
changeDat[['scr_id', 'med_cond','afterTrt', 'monthFU','ThreeMonths']].groupby(['med_cond']).describe(percentiles=[0.025,0.975]) # count how many in each group

In [ ]:
columns = ['scr_id', 'med_cond','beforeTrt','afterTrt','monthFU', 'ThreeMonths']
changeDat_melt = pd.melt(changeDat[columns], 
                id_vars=['scr_id','med_cond'], 
                         var_name = 'time', value_name='pcl')
changeDat_melt.head()
#changeDat_melt.to_csv('changeMelt.csv', index=False)

In [ ]:

px.line(x='time', y='pcl', facet_col= 'med_cond', 
           data_frame=changeDat_melt, color='scr_id')

In [ ]:
changeDat_melt['time'] = changeDat_melt.time.astype('category')
data = changeDat_melt[changeDat_melt.time!='beforeTrt']
data.to_csv('meltedPCL.csv', index=False)

In [ ]:
data

In [ ]:
# simple linear regression of time * pcl (no random effect)
import pymc3 as pm
from pymc3.glm import GLM

with pm.Model() as model_glm:
    GLM.from_formula('pcl ~ time', data = changeDat_melt)
    trace = pm.sample(draws=2000, tune=2000, cores=5)

In [ ]:
pm.summary(trace)

In [ ]:
## Use actual values and not delta
import numpy as np
fullPCL_melt['scr_id'] = fullPCL_melt.scr_id.astype('category')
n_subs = len(np.unique(fullPCL_melt.scr_id))
fullPCL_melt.time = pd.Categorical(fullPCL_melt.time, 
                    categories=['visit_1','visit_7','30_days', '90_days'])
timeIDX = fullPCL_melt.time.cat.codes.values
#fullPCL_melt
print(timeIDX)
#fullPCL_melt
fullPCL_melt.med_cond = fullPCL_melt.med_cond.astype('category')
medIDX = fullPCL_melt.med_cond.cat.codes.values
medIDX

In [ ]:
fullPCL[['med_cond', 'visit_7', '30_days']].groupby('med_cond').describe()

In [ ]:
# multilevel model with reparametarization
# the offset specification is taken from https://twiecki.io/blog/2017/02/08/bayesian-hierchical-non-centered/
with pm.Model() as multilevel_real:
    # Hyperpriors for group nodes
    sigma_a = pm.HalfCauchy('sigma_a', 5)
    mu_a = pm.Normal('mu_a', mu=0, sd=20)
    mu_b = pm.Normal('mu_b', mu=0, sd=20)
    sigma_b = pm.HalfCauchy('sigma_b', 5)

   
   # a_offset = pm.Normal('a_offset', mu=0, sd=1, shape=n_subs)
    #a = pm.Deterministic('a', mu_a + a_offset * sigma_a)
    a = pm.Normal('a', mu=20, sd=20, shape=n_subs)
    #z = pm.Normal('z', mu = 0, sigma = 1, shape = n_subs)
       
    b = pm.Normal('b', mu=0, sd=20, shape=4)
    # b_offset = pm.Normal('b_offset', mu=0, sd=1 , shape = 4)
    #b = pm.Deterministic('b', mu_b + b_offset * sigma_b)   
    inter = pm.Normal('inter', mu=0, sd=5) # modeling interaction for each treatment
    # set another prior to the group (i..e medication condition)
    
    bMed = pm.Normal('bMed', mu=0, sd=20, shape=2)
    # Model error
    eps = pm.HalfCauchy('eps', 10)

    y_hat = a[sub_idx] + b[timeIDX]*timeIDX + bMed[medIDX]*fullPCL_melt.med_cond + inter*timeIDX*medIDX

    # Data likelihood
    likelihood = pm.Normal('likelihood', mu=y_hat,
                           sigma=eps, observed=fullPCL_melt.pcl)
    real_trace = pm.sample(4000, tune=2000, target_accept=.9)

In [ ]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(pm.waic(real_trace))
pm.summary(real_trace, credible_interval=.95, round_to=3)


In [ ]:
pm.traceplot(real_trace)

In [ ]:
## these results lookd like the lme4 results in R
# lets run WAIC
pm.stats.waic(trace)

In [ ]:
# random effect Bayesian model for symptoms
import numpy as np
# set data
# set indexing for subjects
changeDat_melt['scr_id'] = changeDat_melt.scr_id.astype('category')
sub_idx = changeDat_melt.scr_id.cat.codes.values # get county category index values (613 x 1) vec with nums [0-7]
print(sub_idx)
# set y observerd
y_obs = changeDat_melt.pcl.values
#print(data.pcl)
n_subs = len(np.unique(changeDat_melt.scr_id))
print(n_subs)
# set time as index also
# set only three categories (after, 1month, 3months)
changeDat_melt.time = pd.Categorical(changeDat_melt.time, 
                    categories=['beforeTrt','afterTrt','monthFU', 'ThreeMonths'])
timeIDX = changeDat_melt.time.cat.codes.values
print(timeIDX)

In [ ]:
# This model just uses the time as main effect (no checking per timepoint)
# So similar to simple mixed models with subjects as the only random effect
with pm.Model() as hierarchical_model:
  
    a = pm.Normal('a', mu=0, sd=20, shape=n_subs)
    # Intercept for each timepoint, distributed around group mean mu_a
    b = pm.Normal('b', mu=0, sd=20)
    
    # set another prior to the group (i..e medication condition)
    bMed = pm.Normal('bMed', mu=0, sd=20)
   
    # Model error
    eps = pm.HalfCauchy('eps', 10)

    y_hat = a[sub_idx] + b*timeIDX + bMed*fullPCL_melt.med_cond 

    # Data likelihood
    likelihood = pm.Normal('likelihood', mu=y_hat,
                           sigma=eps, observed=fullPCL_melt.pcl)
    hierarchical_trace = pm.sample(4000, tune=2000, target_accept=.9)

In [ ]:
print(pm.stats.waic(hierarchical_trace))
pm.summary(hierarchical_trace)

In [ ]:
pm.traceplot(hierarchical_trace)

In [ ]:
# multilevel model with reparametarization
with pm.Model() as multiLevel_repar:
    # Hyperpriors for group nodes

    sigma_a = pm.HalfNormal('sigma_a', 5.)

    a = pm.Normal('a', mu=0, sigma=10)
    z = pm.Normal('z', mu = 0, sigma = 1, shape = n_subs)
    
    
    mu_b = pm.Normal('mu_b', mu=0, sigma=5)
    b = pm.Normal('b', mu=mu_b, sigma=5,shape=4)
    # set another prior to the group (i..e medication condition)
    
    bMed = pm.Normal('bMed', mu=0, sigma=10)
    # Model error
    eps = pm.HalfCauchy('eps', 10)

    y_hat = a + z[sub_idx]*sigma_a + b[timeIDX]*timeIDX + bMed*changeDat_melt.med_cond 

    # Data likelihood
    likelihood = pm.Normal('likelihood', mu=y_hat,
                           sigma=eps, observed=changeDat_melt.pcl)
    multiLevel_trace = pm.sample(4000, tune=2000, target_accept=.9)

In [ ]:
pm.summary(multiLevel_trace, credible_interval=.95)
#pm.model_to_graphviz(multiLevel_repar)

In [ ]:
with pm.Model() as prior_GLMmodel:
    a = pm.Normal('a', mu=0, sigma=10)#, shape=4)
    # Intercept for each timepoint, distributed around group mean mu_a
    b = pm.Normal('b', mu=0, sigma=10, shape=4)
    
    # set another prior to the group (i..e medication condition)
    bMed = pm.Normal('bMed', mu=0, sigma=5)
    # Model error
    eps = pm.HalfCauchy('eps', 10)

    y_hat = a + b[timeIDX]*timeIDX + bMed*changeDat_melt.med_cond 

    # Data likelihood
    likelihood = pm.Normal('likelihood', mu=y_hat,
                           sigma=eps, observed=changeDat_melt.pcl)
    
    priorGLM_trace = pm.sample(4000, tune=2000)

In [ ]:
pm.summary(priorGLM_trace)

In [ ]:
with hierarchical_model:
    hierarchical_trace = pm.sample(4000, tune=2000, target_accept=.9)

In [ ]:
pm.summary(hierarchical_trace, round_to=2, credible_interval=.95)

### Using intercept as random variable
- using the subjects as different intercept

In [ ]:
p = pm.waic(hierarchical_trace, hierarchical_model)
g = pm.waic(trace, model_glm)
print(p)
print(g)

In [ ]:
pm.compare({'hierarchical': hierarchical_trace, 'glm': trace, 'priorGLM': priorGLM_trace,
           'multilevel':multiLevel_trace})

In [ ]:
# compare LOO
df_comp_LOO = pm.compare({'hierarchical': hierarchical_trace, 'glm': trace,
                         'priorGLM': priorGLM_trace, 'multilevel':multiLevel_trace}
                         , ic='LOO')
df_comp_LOO

In [ ]:
pm.plot_compare(df_comp_LOO)

In [ ]:
#pm.forestplot(trace, var_names=[], credible_interval=.95)
g1 = pm.plot_forest(hierarchical_trace,  credible_interval=.95)


In [ ]:
## Build nice dataframe from the statistical model for plotting

dfStats = pd.DataFrame({'beforeTreatment': multiLevel_trace['b'][-1000:,0],
                       'afterTreatment':multiLevel_trace['b'][-1000:,1],
                       'One-Month_FollowUp': multiLevel_trace['b'][-1000:,2],
                       'Three-Month_FulloUp': multiLevel_trace['b'][-1000:,3]})

# melt it
dfStats_melt = pd.melt(dfStats, var_name = 'time', value_name='pcl')
dfStats

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Changes in PCL scores")
g1 = sns.boxplot(x='time',y='pcl', data=dfStats_melt, showfliers = False)
g1.set_xlabel("")
g1.set_xticklabels(["Before Treatment", 'After Treatment', '30 Days Follow-Up', '90 Days Follow-Up'],
                  fontsize = 14)
g1.set_ylabel("Symptoms (PCL-5 Score)", fontsize = 14)
plt.show()


### so, the best model is the one that takes different times, but from same distribution.  not the general one and not one that takes time as constant (obviously)

## Grab demographic information (sex, age)

In [ ]:
print(list(allDat.columns)) # listing all variables in df

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w